## Setup

Run using docker by doing:

```
>>> docker-compose up -d
```

In [1]:
(require '[clojupyter.misc.helper :as helper])

nil

In [2]:
(->> '[[cnuernber/libpython-clj "1.2"]
       [panthera "0.1-alpha.16"]]
     (map helper/add-dependencies)
     doall)

:done

:done

In [67]:
(ns panthera-test
  (:require
   [libpython-clj.python :as py]
   [panthera.panthera :as pt]
   [panthera.numpy :as np]
   [panthera.pandas.utils :as u]
   [clojure.pprint :refer [pprint]]
   [clojure.repl :refer [doc]]))

nil

In [4]:
(py/initialize!)

:ok

## Wrapping Scikit Learn (sklearn)

In [ ]:
(defonce sk (py/import-module "sklearn"))

(py/run-simple-string "from sklearn import *")

(defn sklearn
  ([k] ((module sk) k))
  ([k args] (((module sk) k) args)))

In [ ]:
(def sample-data (sklearn [:datasets :make-multilabel-classification]
                          {:random-state 0 }))

In [ ]:
(def X (first sample-data))

In [ ]:
(doc py/call-attr)

In [ ]:
(py/python-type X)

In [ ]:
(pprint (take 10 X))

In [ ]:
(def decomp ((sklearn [:decomposition])))

In [ ]:
(def lda (-> ((sklearn [:decomposition]))
             (py/get-attr "LatentDirichletAllocation")))

## Wrapping Gensim

In [6]:
(defonce gs (py/import-module "gensim"))
(py/run-simple-string "from gensim.test.utils import *")

:done

:done

In [7]:
(pprint (py/att-type-map gs))

{"__builtins__" :dict,
 "__cached__" :str,
 "__doc__" :str,
 "__file__" :str,
 "__loader__" :source-file-loader,
 "__name__" :str,
 "__package__" :str,
 "__path__" :list,
 "__spec__" :module-spec,
 "__version__" :str,
 "_matutils" :module,
 "corpora" :module,
 "interfaces" :module,
 "logger" :logger,
 "logging" :module,
 "matutils" :module,
 "models" :module,
 "parsing" :module,
 "scripts" :module,
 "similarities" :module,
 "summarization" :module,
 "test" :module,
 "topic_coherence" :module,
 "utils" :module}


nil

In [68]:
gensim.models.LdaModel


(defn gensim
  ([k] ((np/module gs) k))
  ([k args] (((np/module gs) k) args)))

#'panthera-test/gensim

In [105]:
(def common-texts (gensim [:test :utils :common-texts]))

#'panthera-test/common-texts

In [107]:
(common-texts)

[['human', 'interface', 'computer'], ['survey', 'user', 'computer', 'system', 'response', 'time'], ['eps', 'user', 'interface', 'system'], ['system', 'human', 'system', 'eps'], ['user', 'response', 'time'], ['trees'], ['graph', 'trees'], ['graph', 'minors', 'trees'], ['graph', 'minors', 'survey']]

In [108]:
(def Dictionary (-> (py/get-attr gs "corpora")
                    (py/get-attr "dictionary")
                    (py/get-attr "Dictionary")))

#'panthera-test/Dictionary

In [109]:
(def common-dictionary (Dictionary (common-texts)))

#'panthera-test/common-dictionary

In [72]:
(type common-dictionary)

:pyobject

In [74]:
(def common-corpus (for [t (common-texts)] (py/call-attr common-dictionary "doc2bow" t)))

#'panthera-test/common-corpus

In [110]:
(map type common-corpus)

(:pyobject :pyobject :pyobject :pyobject :pyobject :pyobject :pyobject :pyobject :pyobject)

In [75]:
(type common-corpus)

clojure.lang.LazySeq

In [101]:
common-corpus

([(0, 1), (1, 1), (2, 1)] [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)] [(2, 1), (5, 1), (7, 1), (8, 1)] [(1, 1), (5, 2), (8, 1)] [(3, 1), (6, 1), (7, 1)] [(9, 1)] [(9, 1), (10, 1)] [(9, 1), (10, 1), (11, 1)] [(4, 1), (10, 1), (11, 1)])

In [83]:
(def gensim-models (gensim [:models]))

#'panthera-test/gensim-models

In [86]:
(def LdaModel (py/get-attr (gensim-models) "LdaModel"))

#'panthera-test/LdaModel

In [104]:
(type common-corpus)

clojure.lang.LazySeq

In [113]:
(py/call-attr (gensim-models) "LdaModel" common-corpus)

;; For loop of python dictionaries might be a good minimal example for Chris.

LdaModel(num_terms=12, num_topics=100, decay=0.5, chunksize=2000)

In [114]:
(doc py/call-attr-kw)

-------------------------
libpython-clj.python/call-attr-kw
([item att-name arglist kw-map])


nil

In [118]:
(py/call-attr-kw (gensim-models) "LdaModel" common-corpus { "num_topics" 5})

Execution error (ExceptionInfo) at libpython-clj.python.interpreter/check-error-throw (interpreter.clj:261).
TypeError: __init__() got multiple values for argument 'num_topics'



class clojure.lang.ExceptionInfo: 

In [115]:
(doc py/call-attr-kw)

-------------------------
libpython-clj.python/call-attr-kw
([item att-name arglist kw-map])


nil